In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
from ansys.mapdl.core import launch_mapdl
from ansys.mapdl import reader as pymapdl_reader

In [2]:
import pyvista
# 기본적으로 plot이 안돼서 html로 plot하도록 만듦
pyvista.set_jupyter_backend('html')
pyvista.start_xvfb()

In [3]:
# Start an MAPDL instance
try:
    mapdl = launch_mapdl(start_instance=False, port=50052)
except:
    mapdl = launch_mapdl(start_instance=True, port=50052)

In [54]:
mapdl.finish()
mapdl.clear()
mapdl.prep7()

*** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2024 R2          24.2     ***
 Ansys Mechanical Enterprise Academic Research     
 01078600  VERSION=LINUX x64     17:47:34  DEC 01, 2024 CP=    768.011

                                                                               



          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****

## Sine Kagome Lattice Creation

In [30]:
design_width = 60/1000
design_height = 25/1000
total_width = 144/1000
total_z = 5/1000
# density = 0.5

n_cell = 3
relative_density = 0.5

In [31]:
wall_thickness = 1/1000
amplitude = 0.5/1000

cell_size = (design_height - 2*amplitude - wall_thickness)/np.sqrt(3)/n_cell
cell_size

0.004426352063787131

In [ ]:
mapdl.btol(1.0e-8)

BOOLEAN TOLERANCE (NON-DEFAULT) : 

      THE POINT COINCIDENCE TOLERANCE     0.1000000000E-06 WILL BE USED

In [50]:
resolution = 20

def rotate(x, y, theta):
    return x*np.cos(theta)-y*np.sin(theta), x*np.sin(theta)+y*np.cos(theta)

def draw_sin(startx, starty, theta):
    for i in range(resolution+1):
        x = cell_size/resolution*i
        y = np.sin(cell_size/resolution*i/cell_size*2*np.pi)*amplitude
        y_prime = amplitude*2*np.pi/cell_size*np.cos(2*np.pi*x/cell_size)

        dy = abs(wall_thickness/2/np.sqrt(1+1/y_prime**2)*(-1/y_prime))
        dx = -y_prime*dy

        ku1 = mapdl.k(x=startx+rotate(x+dx,y+dy,theta)[0], y=starty+rotate(x+dx,y+dy,theta)[1])
        kd1 = mapdl.k(x=startx+rotate(x-dx,y-dy,theta)[0], y=starty+rotate(x-dx,y-dy,theta)[1])

        if i != 0:
            a1 = mapdl.a(ku1, kd1, kd0, ku0)

        if i >= 2:
            a0 = mapdl.aadd(a0, a1)
        elif i == 1:
            a0 = a1

        ku0 = ku1
        kd0 = kd1
    return a0

In [51]:
for i in range(n_cell+1):
    j = 0
    area_list = []
    while True:
        startx = cell_size*j - np.sqrt(3)/2*wall_thickness
        starty = np.sqrt(3)*cell_size*i + amplitude + wall_thickness/2
        if startx > design_width+wall_thickness:
            break
        area_list.append(draw_sin(startx, starty, 0))
        j += 1
    for i, area in enumerate(area_list):
        if i == 0:
            a0 = area
            continue
        a0 = mapdl.aadd(area, a0)


In [52]:
for i in range(n_cell):
    j = 0
    while True:
        if cell_size*(2*j+1-i%2) - np.sqrt(3)/2*wall_thickness > design_width+wall_thickness:
            break
        a1 = draw_sin(cell_size*(2*j+1-i%2) - np.sqrt(3)/2*wall_thickness, np.sqrt(3)*cell_size*i + amplitude + wall_thickness/2, np.pi/3)
        a2 = draw_sin(cell_size*(2*j+2-i%2) - np.sqrt(3)/2*wall_thickness, np.sqrt(3)*cell_size*i + amplitude + wall_thickness/2, np.pi*2/3)
        a3 = draw_sin(cell_size*(2*j+1-i%2) - np.sqrt(3)/2*wall_thickness, np.sqrt(3)*cell_size*(i+1) + amplitude + wall_thickness/2, -np.pi/3)
        a4 = draw_sin(cell_size*(2*j+2-i%2) - np.sqrt(3)/2*wall_thickness, np.sqrt(3)*cell_size*(i+1) + amplitude + wall_thickness/2, -np.pi*2/3)
        mapdl.aadd(a1, a2, a3, a4)
        j += 1

In [53]:
mapdl.rectng(x1=-total_width/2+design_width/2, x2=0, y1=0, y2=design_height)
mapdl.rectng(design_width, total_width/2+design_width/2, 0, design_height)
mapdl.aadd("all")

30

In [55]:
from functions import kagome_sine_horizontal

In [56]:
kagome_sine_horizontal(mapdl, n_cell=2, relative_density=0.4, amplitude_ratio=0.5, three_dim=False)

0.0017095799285235158

In [57]:
mapdl.aplot()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [16]:
stress_strain = pd.read_excel('PLA stress-strain.xlsx')
stress_strain = stress_strain.iloc[:8]  # due to limitations of MISO model
stress_strain['y'] = stress_strain['y (MPa)']*1e6
stress_strain['x'] = stress_strain['x']/100
UTS = stress_strain['y'].max()
Youngs_Modulus = stress_strain['y'][1]/stress_strain['x'][1]
max_plastic_strain = stress_strain['x'].iloc[-1]

mapdl.mp('EX', 1, Youngs_Modulus)   # Young's Modulus in Pascals
mapdl.mp('PRXY', 1, 0.33)   # Poisson's Ratio

# Non linear 부분 정의
stress_strain['x'] = stress_strain['x'] - stress_strain['y']/Youngs_Modulus
mapdl.tb('PLASTIC', 1, '', 'MISO')
for i in range(len(stress_strain)):
    mapdl.tbpt(i, stress_strain['x'][i], stress_strain['y'][i])

In [17]:
mapdl.allsel()

SELECT ALL ENTITIES OF TYPE= ALL  AND BELOW

## Simulation

In [ ]:
mapdl.parameters.keys()

## Saving 3D Object

In [1]:
import os

current_dir = os.getcwd()

In [19]:
mapdl.cdwrite(fname=f"{current_dir}/result/Kagome_3", ext = "txt")

*** MAPDL GLOBAL STATUS *** 

 TITLE =                                                                       
       504 KEYPOINTS DEFINED     MAX KEYPOINT NUMBER =            504
       756 LINES DEFINED         MAX LINE NUMBER =                756
       254 AREAS DEFINED         MAX AREA NUMBER =                254
         1 VOLUMES DEFINED       MAX VOLUME NUMBER =                1
         0 ELEM TYPES DEFINED    MAX ELEM TYPE NUMBER =             0
         0 ELEMENTS DEFINED      MAX ELEMENT NUMBER =               0
         0 NODES DEFINED         MAX NODE NUMBER =                  0

 WRITE ANSYS DATABASE AS AN ANSYS INPUT FILE: /home/mjk32/IIAD_pyMAPDL/result/Kagome_3.txt                                                                                                                                                                                                                        

 WRITE IGES FILE= /home/mjk32/IIAD_pyMAPDL/result/Kagome_3.iges

 ATTRIBUTES WILL BE WRITTE

In [20]:
mapdl.exit()

In [4]:
result = pymapdl_reader.read_binary(f"{current_dir}/result/temp_triangular.rst")

FileNotFoundError: /home/mjk32/IIAD_pyMAPDL/result/temp_triangular.rst is not a file or cannot be found